In [2]:
#Imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, timedelta # date-time formatting
import os


# Importing Data

In [3]:
#importing data into df and reading available columns

#KAGGLE IMPORTS
#dfwine = pd.read_csv('../input/california-wine-production-19802020/Californa_Wine_Production_1980_2020.csv')
#dffire = pd.read_csv('../input/california-wildfire-incidents-20132020/California_Fire_Incidents.csv')

#LOCAL IMPORTS
df1 = pd.read_csv('Californa_Wine_Production_1980_2020.csv')
df2 = pd.read_csv('California_Fire_Incidents.csv')

#Reading Column Names
print("Wine column names", df1.columns)
print()
print("Fire column names", df2.columns)

Wine column names Index(['Year', 'CommodityCode', 'CropName', 'CountyCode', 'County',
       'HarvestedAcres', 'Yield(Unit/Acre)', 'Production',
       'Price(Dollars/Unit)', 'Unit', 'Value(Dollars)'],
      dtype='object')

Fire column names Index(['AcresBurned', 'Active', 'AdminUnit', 'AirTankers', 'ArchiveYear',
       'CalFireIncident', 'CanonicalUrl', 'ConditionStatement',
       'ControlStatement', 'Counties', 'CountyIds', 'CrewsInvolved', 'Dozers',
       'Engines', 'Extinguished', 'Fatalities', 'Featured', 'Final',
       'FuelType', 'Helicopters', 'Injuries', 'Latitude', 'Location',
       'Longitude', 'MajorIncident', 'Name', 'PercentContained',
       'PersonnelInvolved', 'Public', 'SearchDescription', 'SearchKeywords',
       'Started', 'Status', 'StructuresDamaged', 'StructuresDestroyed',
       'StructuresEvacuated', 'StructuresThreatened', 'UniqueId', 'Updated',
       'WaterTenders'],
      dtype='object')


# Cleaning Fire Data

In [4]:
#Keeping only needed columns
dffire = df2[['AcresBurned',
              'ArchiveYear',
              'Counties',
              'CountyIds',
              'Started',
              'Extinguished',
              'StructuresDestroyed']]

#Renaming Counties to County
dffire = dffire.rename(columns={'Counties':'County'})

#Cleaning Time Format
dffire['Started'] = dffire['Started'].str.slice(stop=10) + " " + dffire['Started'].str.slice(start=11,stop=19)
dffire['Extinguished'] = dffire['Extinguished'].str.slice(stop=10) + " " + dffire['Extinguished'].str.slice(start=11,stop=19)

#Formatting Time to datetime
dffire['Started'] = pd.to_datetime(dffire['Started'], format = "%Y-%m-%d %H:%M:%S")
dffire['Ended'] = pd.to_datetime(dffire['Extinguished'], format = "%Y-%m-%d %H:%M:%S")

#Dropping NA values
dffire = dffire.dropna(subset=['Extinguished'])


In [5]:
#Calculation Duration of fire
dffire['Duration'] = abs(dffire['Started'] - dffire['Ended']) 

#Summing fire Duration and AcresBurned by county and year
dffireD = dffire.groupby(['County','ArchiveYear'])[['Duration']].sum().reset_index()
dffireA = dffire.groupby(['County','ArchiveYear'])[['AcresBurned']].sum().reset_index()
dffire3 = dffireD.merge(dffireA,on=['County','ArchiveYear'])


dffire3 = dffire3.rename(columns={'ArchiveYear':'Year'})


# Cleaning Wine Data

In [6]:
#Cleaning 
dfwine = df1.drop(columns = ['Unit','CropName'])
dfwine = df1[['Year',
              'County',
              'HarvestedAcres',
              'Yield(Unit/Acre)',
              'Production',
              'Price(Dollars/Unit)',
              'Value(Dollars)']]



dfwine = dfwine[dfwine['Year'] > 2012] 

#showing all UNIQUE entires for COUNTY
#df_clean.county.unique
#df_clean['county']

# Merging the Data

In [7]:
#Merging data on County and Year
dfmain = dfwine.merge(dffire3,on=['County','Year'])



# Saving to CSV

In [10]:
#Saving to csv
dfmain.to_csv('WineProductionAndWildFireStatistics.csv',index=False) # save to notebook output
